In [31]:
from google.colab import files
uploded = files.upload()

Saving train2.csv to train2 (2).csv


In [0]:
import pandas as pd
import io
train = pd.read_csv(io.BytesIO(uploded['train2.csv']))


In [35]:
from google.colab import files
uploaded = files.upload()

Saving test2.csv to test2 (2).csv


In [0]:
import pandas as pd
import io
test = pd.read_csv(io.BytesIO(uploaded['test2.csv']))


In [64]:
train.head()

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1


In [38]:
train.text[1]

'I can completely understand why you’d want to try it. But, results reported in lectures don’t always stand up to the scrutiny of peer-review during publication. There so much still to do before this is convincing. I hope that it does work out, I really do. And if you’re aware of and happy with the risks, then that’s great. I just think it’s important to present this in a balanced way, and to understand why we don’t move straight from the first show of promise in an animal study to using drugs on humans. There’s still a lot of animal data to gather, and human data to gather before anyone can tell if it’s safe or effective. I can’t tell you how many times animal studies don’t follow through to humans, but it’s one of the major attrition points in drug development. You’ve been through some of the unpredictability issues with Cladribine/Gilenya, where there was an interaction that wasn’t predicted. But once people try it, the doctors can see patterns and work out what’s going on. Clemasti

In [0]:
x_train = train['text']

In [0]:
y_train=train['sentiment']

In [0]:
x_test=test['text']

In [93]:
test.head()

,unique_hash,text,drug
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod
2,50b6d851bcff4f35afe354937949e9948975adf7,Apparently it's shingles! :-/ I do have a few ...,humira
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,If the Docetaxel doing once a week x3 weeks th...,tagrisso
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,"CC, Stelara worked in a matter of days for me....",stelara


In [66]:
y_train.head()

0    2
1    2
2    2
3    2
4    1
Name: sentiment, dtype: int64

In [84]:
print(x_train_bow.shape, y_train.shape)

(3, 3) (5279,)


In [85]:
tfidfvectore = TfidfVectorizer(ngram_range=(1,1))
tfidfvectore.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
X_train_tfidf = tfidfvectore.transform(x_train)

In [87]:
X_train_tfidf.shape

(5279, 41015)

In [0]:
from sklearn.metrics import roc_curve, auc
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
train_auc = []
from sklearn.feature_extraction.text import CountVectorizer

sgd = SGDClassifier(loss='hinge',alpha=10, penalty='l2')
sgd.fit(X_train_tfidf, y_train);

In [100]:
from sklearn.model_selection import GridSearchCV
alphas =[10**-4, 10**-3,10**-2,10**-1,10**0,10,10**2,10**3,10**4]

param_grid = {'alpha':alphas}
grid = GridSearchCV(estimator = sgd,param_grid=param_grid ,scoring = 'f1_macro')
grid.fit(X_train_tfidf, y_train)


print(grid.best_estimator_)
print(grid.score(X_train_tfidf, y_train))


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
0.8605788464799748


In [103]:
sgd = SGDClassifier(loss='hinge',alpha=0.0001, penalty='l2')
sgd.fit(X_train_tfidf, y_train);
calibrated = CalibratedClassifierCV(sgd, method='sigmoid',cv=5)
calibrated.fit(X_train_tfidf, y_train)

CalibratedClassifierCV(base_estimator=SGDClassifier(alpha=0.0001, average=False,
                                                    class_weight=None,
                                                    early_stopping=False,
                                                    epsilon=0.1, eta0=0.0,
                                                    fit_intercept=True,
                                                    l1_ratio=0.15,
                                                    learning_rate='optimal',
                                                    loss='hinge', max_iter=1000,
                                                    n_iter_no_change=5,
                                                    n_jobs=None, penalty='l2',
                                                    power_t=0.5,
                                                    random_state=None,
                                                    shuffle=True, tol=0.001,
                                      

In [106]:
print("="*100)

from sklearn.metrics import confusion_matrix

print("Train confusion matrix")
print(confusion_matrix(y_train, calibrated.predict(X_train_tfidf)))


Train confusion matrix
[[ 152    2  463]
 [   0  406  431]
 [   0    3 3822]]
